# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300, 500],
         'max_depth': [30, 60, 90, 120, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
14,46.610498,0.360711,0.696438,0.086710,120,300,"{'max_depth': 120, 'n_estimators': 300}",0.978456,0.977558,0.975741,0.968553,0.973046,0.974671,0.003574,1
19,50.943764,6.323530,0.421152,0.071536,None,500,"{'max_depth': None, 'n_estimators': 500}",0.978456,0.976661,0.975741,0.969452,0.970350,0.974132,0.003574,2
15,74.382362,1.370521,0.916652,0.254768,120,500,"{'max_depth': 120, 'n_estimators': 500}",0.979354,0.977558,0.975741,0.967655,0.970350,0.974132,0.004424,3
18,43.939478,2.020208,0.529729,0.121155,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.978456,0.974843,0.967655,0.972147,0.973952,0.003777,4
13,26.355893,0.719877,0.547535,0.121972,120,150,"{'max_depth': 120, 'n_estimators': 150}",0.979354,0.976661,0.974843,0.968553,0.970350,0.973952,0.003986,5


In [7]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300, 500],
         'max_depth': [30, 60, 90, 120, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count, data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
14,19.278460,0.140127,0.193745,0.007645,120,300,"{'max_depth': 120, 'n_estimators': 300}",0.977558,0.974865,0.973046,0.967655,0.964960,0.971617,0.004646,1
15,31.716129,0.399313,0.300719,0.023240,120,500,"{'max_depth': 120, 'n_estimators': 500}",0.977558,0.972172,0.973046,0.967655,0.966757,0.971438,0.003919,2
10,17.397877,0.078324,0.188617,0.003695,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976661,0.973968,0.972147,0.965858,0.965858,0.970898,0.004359,3
19,22.956086,2.557932,0.202244,0.013538,None,500,"{'max_depth': None, 'n_estimators': 500}",0.977558,0.971275,0.973046,0.966757,0.964960,0.970719,0.004503,4
13,9.881254,0.085462,0.100592,0.001602,120,150,"{'max_depth': 120, 'n_estimators': 150}",0.976661,0.971275,0.973944,0.967655,0.963163,0.970539,0.004740,5
